In [ ]:
import os, h5py, numpy as np
import illustris_python as il
import yt
import yt.units as ytu
from emis import emissivity
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as c
from plot_utils import quiver_from_components
from yt_field_utils import *
from scipy.spatial.transform import Rotation as R

In [ ]:
with h5py.File('mw_tng_sample_catalog.hdf5') as f:
    mw_id = np.array(f['SubfindID'])

In [ ]:
subid = mw_id[5]
outpath = f'/home/cj535/project/Mothra_CGM_CNN/MW_sample/TNG50_snap099_subid{subid:06d}_gas_sphere_500kpc.hdf5'
idx_path = outpath.replace(".hdf5", ".ytindex")
ds = yt.load(outpath, index_filename=idx_path,smoothing_factor=2)


add_emission_fields(ds,line="N2")
add_emission_fields(ds,line="O3")
add_emission_fields(ds,line="Halpha")
#add_emission_weight_fields(ds,minL=1e-43,line='Halpha')
#add_coldgas_weight_fields(ds,maxT=2e4)

vel_bounds = [(-300, -100),(-100, 100),(100, 300)]
for vb in vel_bounds:
    lo,hi = vb
    add_los_mask_fields(ds,lo, hi)
    add_los_mask_emission_fields(ds,lo,hi,line='Halpha')
    add_los_mask_emission_fields(ds,lo,hi,line='N2')
    add_los_mask_emission_fields(ds,lo,hi,line='O3')

In [ ]:
ds.derived_field_list

In [ ]:
angle_number=0
theta_list = [['x',0,'y',0],
              ['x',0,'y',30],
              ['x',0,'y',60],
              ['x',0,'y',90],
              ['x',30,'y',0],
              ['x',60,'y',0],
              ['x',90,'y',0],
              ['x',90,'z',30],
              ['x',90,'z',60],
              ['x',45,'z',45]]
theta = theta_list[angle_number]
r1 = R.from_euler(theta[0], theta[1],degrees=True)
r2 = R.from_euler(theta[2], theta[3],degrees=True)
z_hat = np.array([0,0,1])
y_hat = np.array([0,1,0])
w_hat = r2.apply(r1.apply(z_hat))
v_hat = r2.apply(r1.apply(y_hat))
w_hat /= np.sqrt(np.dot(w_hat,w_hat))
v_hat /= np.sqrt(np.dot(v_hat,v_hat))
u_hat = np.cross(v_hat,w_hat)

In [ ]:
add_velocity_projection_fields(ds,u_hat,v_hat,w_hat)

In [ ]:
width_proper = (500.0, "kpc")                 # proper
center_comov = 0.5 * ds.domain_width          # code center (comoving)
center_prop  = (center_comov.to("kpc") * ds.scale_factor)
sp_small = ds.sphere(center_prop, (50.0, "kpc"))
bv = sp_small.quantities.bulk_velocity()
sp = ds.sphere(center_prop, (500.0, "kpc"))
sp.set_field_parameter("bulk_velocity", bv)
resolution = (256,256)

'''field_list = [("gas","Halpha_brightness"),
              ("gas","Halpha_brightness_-300_-100"),
              ("gas","Halpha_brightness_-100_100"),
              ("gas","Halpha_brightness_100_300")]'''
#field_list = [('gas','Halpha_brightness')]
field_list = [("gas","N2_brightness"),
              ("gas","O3_brightness"),
              ("gas","Halpha_brightness")]

p = yt.OffAxisProjectionPlot(
    ds, w_hat, field_list,
    center=center_prop, width=width_proper,
    north_vector=v_hat,
    method="integrate",
    buff_size=resolution,
    data_source=sp
)
#HalphaL = {}
#range_name = ['bolo','-300_-100','-100_100','100_300']
for i, field in enumerate(field_list):
    #img = p.frb[field]
    #HalphaL[range_name[i]] = img.to_value(img.units)
    p.set_zlim(field, zmin=1e-27, zmax=1e-18)

p.show()